In [ ]:
# Tourist Visitor Information Database for US Cities
### Data Engineering Capstone Project

#### Project Summary
US Federal body for Toursim Management(USATM) has hired my organisation Rusty's Analtics & Data Consulting(RADC) to extract, transform & load immigration and cities data to one table for their team to carry out advanced analytics. USATM has an advanced analytics team that would like to run use cases such as identifying cities with most tourist footfall and then accordingly allocate budgets for marketing/advertising spend for the upcoming tourist season. RADC is an expert in this area and would like to help them out with our expertise in the Data & Analytics area.  

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
We will be reading 2 files - immigration_data_sample.csv and us-cities-demograpgics.csv from S3. Once we have read the 2 files, we will perform Exploratory Data Analysis to review the data and file contents(using the describe function for review). We will then create fact & dimensions table from the data and perform data quality checks once the files are loaded in Redshift. These tables will be available in Redshift for the USATM team to utilise for further insights.   

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
I will be using the udacity provided data sets - immigration_data_sample.csv & us-cities-demographics.csv. Immigration data set contains the list of all inflow to the USA via different modes of transport for a certain period. Other CSV with US cities demographics has resident demographic information for each city in the USA. 

In [11]:
# Read in the data here from S3 bucket
fname = 'immigration_data_sample.csv'
df = pd.read_csv(fname)

#Use Describe function to check basics of the data set. 
#Initial EDA Reveals: 
#Records are from Year 2016
#1000 records in the file
#Data is for the month of April
#75% arrivals are by Air
#Majority of passengers are travelling for 'Pleasure' 
#Average age of travellers is 43, Oldest passenger is 93, Youngest is 1 year old.
#df.describe()
print("First 5 rows")
df.head(5)

First 5 rows


,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.000000000,2016.000000000,4.000000000,209.000000000,209.000000000,HHW,20566.000000000,1.000000000,HI,...,nan,M,1955.000000000,07202016,F,nan,JL,56582674633.000000000,00782,WT
1,2171295,4422636.000000000,2016.000000000,4.000000000,582.000000000,582.000000000,MCA,20567.000000000,1.000000000,TX,...,nan,M,1990.000000000,10222016,M,nan,*GA,94361995930.000000000,XBLNG,B2
2,589494,1195600.000000000,2016.000000000,4.000000000,148.000000000,112.000000000,OGG,20551.000000000,1.000000000,FL,...,nan,M,1940.000000000,07052016,M,nan,LH,55780468433.000000000,00464,WT
3,2631158,5291768.000000000,2016.000000000,4.000000000,297.000000000,297.000000000,LOS,20572.000000000,1.000000000,CA,...,nan,M,1991.000000000,10272016,M,nan,QR,94789696030.000000000,00739,B2
4,3032257,985523.000000000,2016.000000000,4.000000000,111.000000000,111.000000000,CHM,20550.000000000,3.000000000,NY,...,nan,M,1997.000000000,07042016,F,nan,NaN,42322572633.000000000,LAND,WT


In [12]:
print("Print information, shape, and data type for the data frame")
df.info()

Print information, shape, and data type for the data frame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null obj

In [2]:
#Read file from a S3 Bucket
fname = 'us-cities-demographics.csv'
df1 = pd.read_csv(fname, delimiter=';')

#Use this function to convert data with scientific notation to float
pd.set_option('display.float_format', lambda x: '%.9f' % x)

#Use Describe function to check basics of the data set but first.
#Initial EDA Reveals: 
#Total records is 2891
#Average age of regidents in the file is 35
#Max male population is 4.08 million
#Max female population is 4.5 million
#Total population is 8.56 million
#On average there are 9370 veterans across US cities
#Across cities in the US - 40,653 residents are foreign-born
#df1.describe()
df1.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.800000000,40601.000000000,41862.000000000,82463,1562.000000000,30908.000000000,2.600000000,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.000000000,44129.000000000,49500.000000000,93629,4147.000000000,32935.000000000,2.390000000,MA,White,58723
2,Hoover,Alabama,38.500000000,38040.000000000,46799.000000000,84839,4819.000000000,8229.000000000,2.580000000,AL,Asian,4759
3,Rancho Cucamonga,California,34.500000000,88127.000000000,87105.000000000,175232,5821.000000000,33878.000000000,3.180000000,CA,Black or African-American,24437
4,Newark,New Jersey,34.600000000,138040.000000000,143873.000000000,281913,5829.000000000,86253.000000000,2.730000000,NJ,White,76402


In [13]:
print("Print information, shape, and data type for the data frame")
df1.info()

Print information, shape, and data type for the data frame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [16]:
#Count unique values in immigration dataframe
df.nunique()

Unnamed: 0       0
cicid            0
i94yr            0
i94mon           0
i94cit           0
i94res           0
i94port          0
arrdate          0
i94mode          0
i94addr         59
depdate         49
i94bir           0
i94visa          0
count            0
dtadfile         0
visapost       618
occup          996
entdepa          0
entdepd         46
entdepu       1000
matflag         46
biryear          0
dtaddto          0
gender         141
insnum         965
airline         33
admnum           0
fltno            8
visatype         0
dtype: int64

In [17]:
# Checking if any rows in the immigration data frame have missing data
df.isnull().sum()

Unnamed: 0       0
cicid            0
i94yr            0
i94mon           0
i94cit           0
i94res           0
i94port          0
arrdate          0
i94mode          0
i94addr         59
depdate         49
i94bir           0
i94visa          0
count            0
dtadfile         0
visapost       618
occup          996
entdepa          0
entdepd         46
entdepu       1000
matflag         46
biryear          0
dtaddto          0
gender         141
insnum         965
airline         33
admnum           0
fltno            8
visatype         0
dtype: int64

In [25]:
# Fill NaN with a blank space
df['i94addr'] = df['i94addr'].fillna(' ')
df['depdate'] = df['depdate'].fillna(' ')
df['visapost'] = df['visapost'].fillna(' ')
df['occup'] = df['occup'].fillna(' ')
df['gender'] = df['gender'].fillna(' ')
df['airline'] = df['airline'].fillna(' ')
df.isnull().sum()

Unnamed: 0       0
cicid            0
i94yr            0
i94mon           0
i94cit           0
i94res           0
i94port          0
arrdate          0
i94mode          0
i94addr          0
depdate          0
i94bir           0
i94visa          0
count            0
dtadfile         0
visapost         0
occup            0
entdepa          0
entdepd         46
entdepu       1000
matflag         46
biryear          0
dtaddto          0
gender           0
insnum         965
airline          0
admnum           0
fltno            8
visatype         0
dtype: int64

In [21]:
#Check for duplicated data
bool_series = df.duplicated()
print(bool_series)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
970    False
971    False
972    False
973    False
974    False
975    False
976    False
977    False
978    False
979    False
980    False
981    False
982    False
983    False
984    False
985    False
986    False
987    False
988    False
989    False
990    False
991    False
992    False
993    False
994    False
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool


In [15]:
#Count unique values in US Cities dataframe
df1.nunique()
df1.head()

City                       567
State                       49
Median Age                 180
Male Population            593
Female Population          594
Total Population           594
Number of Veterans         577
Foreign-born               587
Average Household Size     161
State Code                  49
Race                         5
Count                     2785
dtype: int64

In [26]:
# Checking if any rows in the us cities demographics data frame have missing data
df1.isnull().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [31]:
# Fill NaN with a mean value
df1['Male Population'] = df1['Male Population'].fillna(df1['Male Population'].mean())
df1['Female Population'] = df1['Female Population'].fillna(df1['Female Population'].mean())
df1['Number of Veterans'] = df1['Number of Veterans'].fillna(df1['Number of Veterans'].mean())
df1['Average Household Size'] = df1['Average Household Size'].fillna(df1['Average Household Size'].mean())
df1['Foreign-born'] = df1['Foreign-born'].fillna(df1['Foreign-born'].mean())
df1.isnull().sum()

City                      0
State                     0
Median Age                0
Male Population           0
Female Population         0
Total Population          0
Number of Veterans        0
Foreign-born              0
Average Household Size    0
State Code                0
Race                      0
Count                     0
dtype: int64

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
Created 3 tables - Immigration, Arrival & State. Please refer to the attachment in my project folder to get a diagrammatic view. I used this model as it would help USATM answer queries such as states with most tourism influx in a year, 
arrival data table would provide insights on days that are particularly popular with tourists and the state table can be used to see the population of a state over a given year and can be expanded
to provide a view on annual basis to assist government agencies to prepare better for immigration inflow.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
1. create_table.py is where I will create my fact and dimension tables for the star schema in Redshift.
2. etl.py is where I will load data from S3 into staging tables on Redshift and then process that data into my analytics tables on Redshift.
3. sql_queries.py is where I have defined my SQL statements, which will be imported into the two other files above.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#1. Run sql_queries.py in the terminal
#2. Run create_tables.py in the terminal
#3. Run ETL.py in the terminal
# Once the above scripts are run, data from S3 buckets would be loaded into Redshift tables for analysis. 
#4. We now have 3 tables(Immigration, State, Arrival) that are ready for further analysis. 


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

# Perform quality checks here
#1. Integrity constraints have been included in the script sql_queries.py under the CREATE table statement. We have used NOT NULL, PRIMARY KEY constraints while creation.
#2. All scripts have been tested and run without any errors. Please refer to the screenshots attached in github.
#3. Source/Count checks on immigration table have revealed 1000 entries in the CSV, 1000 in staging_immi table and 1000 in the final immi table. We are all good. Screenshots captured in github.

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [ ]:
1. Rationale for choice of tools and technologies for the project. 

Q) Why did you choose specific tools & tech?
Answer) We have chosen Amazon suite of products - S3, EC2, Redshift, Jupyter Notebook, Airflow, Python 3. We have used the cloud technologies mainly to deal with the
growing nature of big data and also it allows us to minimise costs by shutting down or going in sleep modes for periods of low demand. Airflow is best of breed
and open source tech. It is widely used in the industry for data orchestration and provides a reliable way to schedule the pipeline as per business requirements. Airflow
also has an active user(developer) community and roll out new features/enhacements regularly.  

Q) Why the chosen schema?
Answer) We have 2 staging tables(staging_immi and staging_state) & 3 transformed tables(immigration, arrival, state). Landing Zone: This has the staging tables and will capture all 
data from source in as-is format and it will have truncate and load strategy. From staging area - data is loaded to the Transform Zone and will have the 3 tables mentioned above.

Q) Is there a trade-off?
Answer) The only trade-off that we can think about is the huge amount of data and the costs associated with that. As the amount of data goes up, it will start to blow up our cloud resources
costs. 

Q) Furnish results of the sample queries mentioned in scope.
Answer) Please refer to the github screenshots(Query 1, Query 2) of 2 queries that were executed on the data.


2. How often should the data be updated and why?

From our engagement with USATM, we found that this information is required on a quarterly basis to ensure
right and fair allocation of funding to boost state specific tourism in the United States. It is hence desired
to update the data every 2nd month of a quarter. 

In [ ]:
3. Write a description of how you would approach the problem differently under the following scenarios:
Q) The data was increased by 100x. 
- Is partitioning required? If so, how
Answer) I have 3 tables that we are using - immigration, arrival and state. We don't expect partition for state as it is a master table and is not expected
to grow drastically. (Please refer to data model on Github)Let's discuss immigration and arrival - we will partition these two tables based on arrival year and arrival month.  

- How can storage be handeled?
Answer) Since we are using arrival year and arrival month as partition columns, this will help further in applying the purging policy and free up storage. We can also
archive the cold data to another storage like aws glacier.


Q) The data populates a dashboard that must be updated on a daily basis by 7am every day. 
- What are the configuration you need to do in identified tool?
Answer) We will be looking for an orchestrating tool(Airflow), which will orchestrate our ETL pipeline and make data available in datawarehouse before 7am. We will schedule the Airflow by updating the CRON entry "0 7 * * *".

- How will this impact business ?
Answer) Airflow is an open source tool so we don't have to pay a license fee. From a business point, there should be minimal impact as we will be putting in alerts
and quality checks(such as Airflow run time & defined SLAs so that critical jobs can finish before 7 AM). 

- What is the operating cost ?
Answer) We intend to operate a medium Managed Workflows environment with Apache Airflow version 2.0.2 in the US East (N. Virginia) region where our variable demand 
requires 10 workers simultaneously for 2 hours a day, we require a total of 3 schedulers to manage our workflow definitions, and retain 40 GB of data 
(approximately 200 daily workflows, each with 20 tasks, stored for 6 months), we would pay the following for the month:

Environment charge
Instance usage (in hours) = 31 days x 24 hrs/day = 744 hours
x $0.74 (price per hour for a medium environment in the US East (N. Virginia) region)
= $ 550.56

Worker charge
Instance usage (in hours) = 31 days x 2 hrs/day x 9 additional instances (10 less 1 included with environment) = 558 hours
x $0.11 (price per hour for a medium worker in the US East (N. Virginia) region)
= $61.38

Scheduler charge
Instance usage (in hours) = 31 days x 24 hrs/day x 1 additional instances (3 less 2 included with environment) = 774 hours
x $0.11 (price per hour for a medium scheduler in the US East (N. Virginia) region)
= $81.84

Meta database charge
40 GB or storage x $0.10 GB-month = $4.00
Total charge = $697.78

- Does the tool require a separate instance ?
Answer) We will spin up a new EC2 instance and our Airflow will be hosted on that VM. We can also reduce cost using Lambda function to build infrstructure on demand. This means that whenever task is completed, we will shutdown our infra to save on costs. 

Q) The database needed to be accessed by 100+ people.
- Does data replication make sense? 
Answer) Considering all of our user base is based out of United States, we will not be proposing a data replication strategy for the short terms. However, if we find
that the data has grown immensely and there is heavy latency then data replication will need to be brought to life. Please note that data replication comes at a cost
and this decision will be based on cost versus benefit analysis.

- What type of replication ?
Answer) We will be opting for log-based incremental replication. Log-based incremental replication is a special case of replication that applies only to database 
sources. This process replicates data based on information from the database log file, which lists changes to the database. This method is the most efficient out of  
the three(Full table replication, Key-based incremental replication), but it must be supported by the source database, as it is by MySQL, PostgreSQL, and Oracle. This 
method works best if the source database structure is relatively static(which is the case for our project).

- Business benefits and impacts on cost?
Answer) Some benefits of data replication are:
a) Improved reliability and availability: If one system goes down due to faulty hardware, malware attack, or another problem, the data can be accessed from a 
    different site.

b) Improved network performance: Having the same data in multiple locations can lower data access latency, since required data can be retrieved closer to 
    where the transaction is executing.

c) Increased data analytics support: Replicating data to a data warehouse empowers distributed analytics teams to work on common projects for business intelligence.

d) Improved test system performance: Data replication facilitates the distribution and synchronization of data for test systems that demand fast data accessibility.

Cost Impacts: Some of the challenges to maintaining consistent data across an organization boil down to limited resources:
a) Money: Keeping copies of the same data in multiple locations leads to higher storage and processor costs.

b) Time: Implementing and managing a data replication system requires dedicated time from an internal team.

c) Bandwidth: Maintaining consistency across data copies requires new procedures and adds traffic to the network.
    
We would need to perform a cost-benefit analysis of the tool the replication strategy and then roll forward.

